## EV example SETAC

copying processes via bw25 didn't work so well as it gave different scores for the process copies despite copying everything (at least I think so) -> see script ...debuggung

instead, I've copied the processes in AB in a bw2 project and now convert it to bw25

In [1]:
import sys
import os
import bw2data as bd
import bw2calc as bc
import bw2io as bi
from bw_temporalis import TemporalDistribution
sys.path.append(os.path.realpath('../'))
import numpy as np


In [2]:
#bi.backup.restore_project_directory(fp = r'C:\Users\MULLERA\brightway2-project-bw25_premise_background_v2-backup.26-March-2024-01-40PM.tar.gz', project_name= "bw25_EV_setac")

In [3]:
import bw2data as bd
PROJECT_NAME = "bw25_EV_setac"
RESET= False

if PROJECT_NAME in bd.projects and not RESET: # use existing project
    print("Using existing project: {}".format(PROJECT_NAME))
    bd.projects.set_current(PROJECT_NAME) 
    
else: # create project from scratch
    print("Creating new project: {}".format(PROJECT_NAME))
    if PROJECT_NAME in bd.projects:
        bd.projects.delete_project(PROJECT_NAME)
    bi.backup.restore_project_directory(r'filepath/to/backup/directory') # tar file shared on slack
    bd.projects.set_current(PROJECT_NAME)
    
bd.databases

Using existing project: bw25_EV_setac


Databases dictionary with 5 object(s):
	biosphere3
	cutoff39
	db_2020
	db_2030
	db_2040

In [4]:
#del bd.databases["foreground"]

In [5]:
bd.projects.twofive

True

In [6]:
# prospective databases were generated with premise, updating only electricity
db_2020 = bd.Database("db_2020")
db_2030 = bd.Database("db_2030")
db_2040 = bd.Database("db_2040")
foreground = bd.Database("foreground")

write foreground db:

In [7]:
#EV transport process 2020
EV_driving_bg = [x for x in db_2020 if (x['name'] == 'transport, passenger car, electric')][0]
EV_driving_bg


'transport, passenger car, electric' (kilometer, GLO, None)

In [8]:
foreground = bd.Database('foreground')
foreground.register()

EV= foreground.new_node(
    code = "EV",
    name = "foreground transport, electric vehicle, 1km",
    location = "GLO",
)
EV.save()

EV.new_edge(input=("foreground", "EV"), amount=1, unit = "kilometer", type="production").save() #add production amount

#add technosphere exchanges
for exc in EV_driving_bg.exchanges(): #looping over the exchanges of the EV process in the background db and adding them to our EV process in the foreground
    if not exc.input == exc.output and "market for passenger car, electric, without battery" not in exc.input["name"]: #skip production exchange of process and car production, which we"ll move to the foreground separately
        #add other exchanges of EV process
        EV.new_edge(input=exc.input.key, amount=exc['amount'], type=exc["type"]).save() #  

In [9]:
# #add car production separately (we skip the ecoinvent GLO market for car production, as it only has 1 input of GLO car production, and no losses)
car_prod_background = [x for x in db_2020 if (x['name'] == 'passenger car production, electric, without battery')][0]

car_prod = foreground.new_node(
    code = "car_prod",
    name = "foreground car production, electric, without battery",
    location = "GLO",
)
car_prod.save()

car_prod.new_edge(input=("foreground", "car_prod"), amount=1, unit= "kilogram", type="production").save() #add production amount
#TODO undertsand why impact of production is different betweeen foreground system and background system
#add technosphere exchanges
for exc in car_prod_background.exchanges(): #looping over the exchanges of the EV process in the background db and adding them to our EV process in the foreground
    if not exc.input == exc.output: #skip production exchange
        #add other exchanges of car production
        car_prod.new_edge(input=exc.input.key, amount=exc['amount'], type=exc["type"]).save() #

 
for exc in EV_driving_bg.exchanges():
    if "market for passenger car, electric, without battery" in exc.input["name"]:
        amount_car_prod_background= exc["amount"] #find amount of car production per 1 km EV driving in background db

EV.new_edge( #add new car prod to EV process
    input=("foreground", "car_prod"),
    amount=amount_car_prod_background,
    type="technosphere"
).save()



Select EV processes in dbs

In [10]:
#EV transport process foreground
EV_driving_fg = [x for x in foreground if (x['name'] == 'foreground transport, electric vehicle, 1km')][0]
EV_driving_fg

'foreground transport, electric vehicle, 1km' (None, GLO, None)

In [11]:
#EV production foreground
car_prod_fg = [x for x in foreground if (x['name'] == 'foreground car production, electric, without battery')][0]
car_prod_fg

'foreground car production, electric, without battery' (None, GLO, None)

In [12]:
# defining Temporal Distributions of system
td_production = TemporalDistribution(
            date=np.array([-1], dtype='timedelta64[Y]'), #2023
            amount=np.array([1]))

td_use_phase = TemporalDistribution( #to be further refined based on use pattern
                    date=np.array([1, 6, 11, 16], dtype='timedelta64[Y]'), #2025, 2030, 2035, 2040
                    amount=np.array([0.25, 0.25, 0.25, 0.25]))

td_eol = TemporalDistribution(
            date=np.array([21], dtype='timedelta64[Y]'), #2045
            amount=np.array([1]))

td_eol_from_production_time = TemporalDistribution(
            date=np.array([22], dtype='timedelta64[Y]'), #2045, counting from 2023
            amount=np.array([1]))


In [13]:
#add TD to electricity and car production
for exc in EV_driving_fg.exchanges():
    if "market for battery, Li-ion"in exc.input["name"]: #battery production
        exc["temporal_distribution"]= td_production
        exc.save()
        print("added TD to ", {exc.input["name"]})

    elif "foreground car production, electric, without battery"in exc.input["name"]: # foreground car production
        exc["temporal_distribution"]= td_production
        exc.save()
        print("added TD to ", {exc.input["name"]})

    elif "market group for electricity, low voltage" in exc.input["name"]: #electricity while driving
        exc["temporal_distribution"]= td_use_phase
        exc.save()
        print("added TD to ", {exc.input["name"]})

    elif "market for used Li-ion battery" in exc.input["name"]:   #battery recycling
        exc["temporal_distribution"]= td_eol
        exc.save()
        print("added TD to ", {exc.input["name"]})

    elif "wear emissions, passenger car" in exc.input["name"]: #brake, road and tyre wear while driving
        exc["temporal_distribution"]= td_use_phase
        exc.save()
        print("added TD to ", {exc.input["name"]})
    
    elif "market for maintenance, passenger car" in exc.input["name"]: #car maintenance while driving
        exc["temporal_distribution"]= td_use_phase
        exc.save()
        print("added TD to ", {exc.input["name"]})
            
    elif exc.input["name"] == "market for road": # road usage while driving
        exc["temporal_distribution"]= td_use_phase
        exc.save()
        print("added TD to ", {exc.input["name"]})
    
    elif exc.input["name"] == exc.output["name"]: #skip production exchange
        continue

    else:
        print("no TD added to ", {exc.input["name"]})

    # car recycling is modelled at the level of car production, and TD is added there

added TD to  {'market for brake wear emissions, passenger car'}
added TD to  {'market for road wear emissions, passenger car'}
added TD to  {'market for used Li-ion battery'}
added TD to  {'market for tyre wear emissions, passenger car'}
added TD to  {'market for road'}
added TD to  {'market for battery, Li-ion, LiMn2O4, rechargeable, prismatic'}
added TD to  {'market for maintenance, passenger car, electric, without battery'}
added TD to  {'market group for electricity, low voltage'}
added TD to  {'foreground car production, electric, without battery'}


In [14]:
for exc in car_prod_fg.exchanges():
    if "waste" in exc.input["name"]: # all waste are EoL processes
        exc["temporal_distribution"]= td_eol_from_production_time 
        exc.save()
        print("added TD to ", {exc.input["name"]})

    elif "manual dismantling" in exc.input["name"]: # all waste are EoL processes
        exc["temporal_distribution"]= td_eol_from_production_time
        exc.save()
        print("added TD to ", {exc.input["name"]})
    
    elif exc.input["name"] == exc.output["name"]: #skip production exchange
        continue

    else:
        print("no TD added to ", {exc.input["name"]})
        # production processes (glider and powertrain) are temporalized at the level of the transport process


added TD to  {'market for waste glass'}
added TD to  {'market for waste rubber, unspecified'}
added TD to  {'market for waste glass'}
added TD to  {'market group for waste glass'}
added TD to  {'market for waste mineral oil'}


added TD to  {'market for waste rubber, unspecified'}
added TD to  {'market for waste glass'}
added TD to  {'market for waste mineral oil'}
added TD to  {'market for waste mineral oil'}
added TD to  {'market for waste rubber, unspecified'}
added TD to  {'market for waste glass'}
added TD to  {'market for waste glass'}
added TD to  {'market for waste glass'}
added TD to  {'market for waste glass'}
no TD added to  {'market for glider, passenger car'}
no TD added to  {'market for powertrain, for electric passenger car'}
added TD to  {'market for manual dismantling of used electric passenger car'}


In [15]:
# checking foreground links
for act in bd.Database('foreground'):
    print(act)
    for exc in act.exchanges():
        if "temporal_distribution" in exc.keys():
            print(exc.input["name"])
            print(exc["temporal_distribution"].date)
            print(exc["temporal_distribution"].amount)
       

'foreground transport, electric vehicle, 1km' (None, GLO, None)
market for brake wear emissions, passenger car
[ 31556952 189341712 347126472 504911232]
[0.25 0.25 0.25 0.25]
market for road wear emissions, passenger car
[ 31556952 189341712 347126472 504911232]
[0.25 0.25 0.25 0.25]
market for used Li-ion battery
[662695992]
[1.]
market for tyre wear emissions, passenger car
[ 31556952 189341712 347126472 504911232]
[0.25 0.25 0.25 0.25]
market for road
[ 31556952 189341712 347126472 504911232]
[0.25 0.25 0.25 0.25]
market for battery, Li-ion, LiMn2O4, rechargeable, prismatic
[-31556952]
[1.]
market for maintenance, passenger car, electric, without battery
[ 31556952 189341712 347126472 504911232]
[0.25 0.25 0.25 0.25]
market group for electricity, low voltage
[ 31556952 189341712 347126472 504911232]
[0.25 0.25 0.25 0.25]
foreground car production, electric, without battery
[-31556952]
[1.]
'foreground car production, electric, without battery' (None, GLO, None)
market for waste glas

select method:

In [16]:
CC_method = [m for m in bd.methods if 'IPCC 2021' in str(m) and  'climate change no LT' in str(m) and 'GWP100' in str(m) and 'no LT' in str(m)][0]
CC_method

('IPCC 2021 no LT',
 'climate change no LT',
 'global warming potential (GWP100) no LT')

In [17]:
demand = {EV_driving_fg.key: 1}
method = CC_method

lca = bc.LCA(demand, method)
lca.lci()
lca.lcia()
lca.score

0.22196471360166709

In [18]:
demand = {EV_driving_bg.key: 1}
method = CC_method

lca = bc.LCA(demand, method)
lca.lci()
lca.lcia()
lca.score

0.23022282296697355

### see notebook ..._debugging for exploration why the scores are different

for now, we simply acceot that the score is different.

### Now medusa LCA from here on:

In [19]:
from datetime import datetime

database_date_dict = {
    "db_2020": datetime.strptime("2020", "%Y"),
    "db_2030": datetime.strptime("2030", "%Y"),
    "db_2040": datetime.strptime("2040", "%Y"),
    "cutoff39": datetime.strptime("2020", "%Y"), # all databases need to have a corresponding time
    "foreground": "dynamic",  # flag databases that should be temporally distributed with "dynamic"
}

In [20]:
SKIPPABLE = [node.id for node in bd.Database('db_2020')] + [
    node.id for node in bd.Database('db_2030')] + [
    node.id for node in bd.Database('db_2040')
    ] 

def filter_function(database_id: int) -> bool:
    return database_id in SKIPPABLE

len(SKIPPABLE)

72315

In [21]:
demand = {('foreground', 'EV'): 1}
method = CC_method

In [23]:
from timex_lca import MedusaLCA
mlca = MedusaLCA(demand, method, None, database_date_dict, max_calc=10) #aborted after 100 min for max_calc = 5000


Starting graph traversal
Calculation count: 3


In [24]:
import warnings
warnings.filterwarnings("ignore")
mlca.build_timeline()

,hash_producer,time_mapped_producer,date_producer,producer,producer_name,hash_consumer,time_mapped_consumer,date_consumer,consumer,consumer_name,amount,interpolation_weights
0,2023,98282,2023-01-01,27559,"market for glider, passenger car",2023,98283,2023-01-01,98281,"foreground car production, electric, without b...",0.912635,"{'cutoff39': 0.6999726252395291, 'db_2030': 0...."
1,2023,98283,2023-01-01,98281,"foreground car production, electric, without b...",2024,98284,2024-01-01,98280,"foreground transport, electric vehicle, 1km",0.006121,"{'cutoff39': 0.6999726252395291, 'db_2030': 0...."
2,2024,98284,2024-01-01,98280,"foreground transport, electric vehicle, 1km",2024,-1,2024-01-01,-1,-1,1.0,"{'cutoff39': 0.6000547495209416, 'db_2030': 0...."
3,2025,98285,2025-01-01,50069,"market group for electricity, low voltage",2024,98284,2024-01-01,98280,"foreground transport, electric vehicle, 1km",0.04975,"{'cutoff39': 0.4998631261976457, 'db_2030': 0...."
4,2030,98286,2030-01-01,50069,"market group for electricity, low voltage",2024,98284,2024-01-01,98280,"foreground transport, electric vehicle, 1km",0.04975,{'db_2030': 1}
5,2035,98287,2035-01-01,50069,"market group for electricity, low voltage",2024,98284,2024-01-01,98280,"foreground transport, electric vehicle, 1km",0.04975,"{'db_2030': 0.5, 'db_2040': 0.5}"
6,2040,98288,2040-01-01,50069,"market group for electricity, low voltage",2024,98284,2024-01-01,98280,"foreground transport, electric vehicle, 1km",0.04975,{'db_2040': 1}


In [25]:
mlca.build_datapackage()
mlca.lci()
mlca.lcia()

UnknownObject: 

In [35]:
# maybe the problem lies in that the background databases have unique reference products on the exchanges, 
# while the foreground processes have ValueError: Can't find a single reference product exchange (found 5 candidates)
# i don't know how to add them

for exc in EV_driving_bg.exchanges():
    print(exc.input["name"], exc.input["database"], exc.input["reference product"], )

for exc in EV_driving_fg.exchanges():
    print(exc.input["name"], exc.input["database"], exc.input["reference product"], )

transport, passenger car, electric db_2020 transport, passenger car, electric
market for brake wear emissions, passenger car db_2020 brake wear emissions, passenger car
market for road wear emissions, passenger car db_2020 road wear emissions, passenger car
market for used Li-ion battery db_2020 used Li-ion battery
market for tyre wear emissions, passenger car db_2020 tyre wear emissions, passenger car
market for road db_2020 road
market for battery, Li-ion, LiMn2O4, rechargeable, prismatic db_2020 battery, Li-ion, LiMn2O4, rechargeable, prismatic
market for maintenance, passenger car, electric, without battery db_2020 maintenance, passenger car, electric, without battery
market for passenger car, electric, without battery db_2020 passenger car, electric, without battery
market group for electricity, low voltage db_2020 electricity, low voltage


KeyError: 

In [36]:
for exc in car_prod_background.exchanges():
    print(exc.input["name"], exc.input["database"], exc.input["reference product"], )

for exc in car_prod_fg.exchanges():
    print(exc.input["name"], exc.input["database"], exc.input["reference product"], )

market for waste glass db_2020 waste glass
market for waste rubber, unspecified db_2020 waste rubber, unspecified
market for waste glass db_2020 waste glass
market group for waste glass db_2020 waste glass
market for waste mineral oil db_2020 waste mineral oil
passenger car production, electric, without battery db_2020 passenger car, electric, without battery
market for waste rubber, unspecified db_2020 waste rubber, unspecified
market for waste glass db_2020 waste glass
market for waste mineral oil db_2020 waste mineral oil
market for waste mineral oil db_2020 waste mineral oil
market for waste rubber, unspecified db_2020 waste rubber, unspecified
market for waste glass db_2020 waste glass
market for waste glass db_2020 waste glass
market for waste glass db_2020 waste glass
market for waste glass db_2020 waste glass
market for glider, passenger car db_2020 glider, passenger car
market for powertrain, for electric passenger car db_2020 powertrain, for electric passenger car
market for 

KeyError: 

In [ ]:
print('Old static LCA Score:', mlca.static_lca.score)
print('New MEDUSA LCA Score:', mlca.score)

In [ ]:
mlca.dynamic_inventory